<table><tr><td height="250" width="500"><img src="https://www.opendataportal.at/wp-content/uploads/2016/09/WU-Wien-Logo.png"/ height="250" width="500"></td><td height="250" width="250"><img src="https://beta.ownyourdata.eu/wp-content/uploads/2018/02/logo512-300x300.png" height="250" width="250"/></td></tr></table>
<hr>
<h1><center>Data Science Lab: Own Your Data </center></h1>
<hr>
<h2><center>Notebook 3: Daily Report </center></h2>

<h3><center>To Do:</center></h3><br>
<ol>
    <li>Find most common locations for each day (more than x data points)</li>
    <li>Find highest speed, longest time between records, longest travel distance for each day </li>
</ol>

In [1]:
import json
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import geopy
import geopy.geocoders
from geopy.geocoders import Nominatim
import datetime

In [2]:
def haversine_np(lat1, lon1, lat2, lon2):
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    m = 6367 * c * 1000
    return (m)

In [3]:
def get_city(lat, lon):
    geolocator = Nominatim(user_agent="just-searching-some-addresses")
    coordinate = ", ".join((str(lat),str(lon)))
    location = geolocator.reverse(coordinate, addressdetails = False, language = "en")
    location_list = str(location).split(", ")
    city = location_list[-3]
    country = location_list[-1]
    return(city, country, location)

In [4]:
def get_coordinates(x):
    geolocator = Nominatim(user_agent="reverse-searching-to-coordinates")
    location = geolocator.geocode(x)
    return(location)

In [5]:
filepath = "./data/user_archive_091118"

In [6]:
with open (filepath, "r") as file:
    all_data = json.load(file)
data = all_data["data"]
for dictionary in data:
    if "Location" in dictionary.keys():
        gps_data = dictionary["Location"]
gps = pd.DataFrame(gps_data)

In [7]:
gps["datetime"] = pd.to_datetime(gps["timestamp"], format="%Y-%m-%dT%H:%M:%S.%fZ")
gps = gps.drop("timestamp", axis=1)

In [8]:
gps["date"] = gps["datetime"].dt.date

In [9]:
records_per_day = gps["date"].groupby(gps["date"]).count()

In [13]:
# find distance_per_day and vistited_places
# to do: stunden zwischen den Records, anzahl der records pro Tag, 
# alle wie viele Minuten durchschnittlich ein Record gemacht wird
# Durchschnittsgeschwindigkeiten
# wenn Distanz größer als 5km: schauen ob vllt verschiedene Orte

gps["last_latitude"] = gps["latitude"].shift(1)
gps["last_longitude"] = gps["longitude"].shift(1)
gps["distance in km"] = haversine_np(lon1 = gps["last_longitude"], 
                                    lat1 = gps["last_latitude"],
                                    lon2 = gps["longitude"],
                                    lat2 = gps["latitude"]) /1000
date_list = []
distance_dict = {}
visited_places = {}
for i in gps["date"]:
    date_list.append(i)
date_list = set(date_list)
for date in date_list:
    coordinates = gps[["latitude", "longitude", "datetime"]].loc[gps["date"] == date]
    coordinates = coordinates.sort_values(by="datetime")
    coordinates = coordinates.reset_index()
    coordinates["last_latitude"] = coordinates["latitude"].shift(1)
    coordinates["last_longitude"] = coordinates["longitude"].shift(1)
    coordinates["distance in km"] = haversine_np(lat1 = coordinates["last_latitude"], lon1 = coordinates["last_longitude"],
                                                lat2 = coordinates["latitude"], lon2 = coordinates["longitude"]) /1000
    coordinates["distance in km"] = coordinates["distance in km"].fillna(0)
    distance_dict[date] = sum(coordinates["distance in km"])
#    Location to append: wenn 2 konsekutive Werte gleich sind oder Datenwert mind. 2 mal vorkommt
#    Find method to address exact row
#    if coordinates[date]["distance in km"] <= 0.050:
#        latlon = (", ").join(coordinates[date]["latitude"], coordinates[date]["longitude"])
#        if latlon not in visited_places:
#            visited_places[latlon] = coordinates[date]["datetime"]
    

KeyError: datetime.date(2018, 10, 11)

In [11]:
# find the visited places per day (at least two consecutive records)
for date in date_list:
    coordinates = gps[["latitude", "longitude"]].loc[gps["date"] == date]
    coordinates.reset_index()
    if 